In [4]:
from fastapi import FastAPI, HTTPException
import httpx
from pydantic import BaseModel

app = FastAPI()

# Configuración de la API de Moodle
MOODLE_URL = "http://localhost/moodle/webservice/rest/server.php"
MOODLE_TOKEN = "7f0aa2b0746edf87b136e976cfaac531"


class CourseData(BaseModel):
    id: int
    fullname: str
    shortname: str
    categoryid: int
    visible: int
    format: str


# Ruta para actualziar un curso en Moodle utilizando el método POST
@app.post("/update_course_moodle")
# Función para actualizar un curso existente en Moodle
async def actualizar_curso( course_data: CourseData):
    # Define la función de la API de Moodle para actualizar cursos
    moodle_api_function = "core_course_update_courses"

    # Configura los parámetros para la solicitud
    params = {
        "wstoken": MOODLE_TOKEN,
        "wsfunction": moodle_api_function,
        "moodlewsrestformat": "json",
        "courses[0][id]": course_data.id,  # ID del curso existente
        "courses[0][fullname]": course_data.fullname,
        "courses[0][shortname]": course_data.shortname,  # Puedes ajustar este parámetro según tus necesidades
        "courses[0][categoryid]": course_data.categoryid, #0,  # ID de la categoría del curso (ajusta según tu estructura)
        "courses[0][visible]": course_data.visible, #1,  # Puedes ajustar la visibilidad del curso (1 para visible, 0 para oculto)
        "courses[0][format]": course_data.format #"topics",  # Puedes ajustar el formato del curso (topics, weeks, etc.)
    }

    # Realiza la solicitud a la API de Moodle
    async with httpx.AsyncClient() as client:
        response = await client.post(MOODLE_URL, params=params)

    # Verifica el estado de la respuesta
    if response.status_code == 200:
        result = response.json()
        if "exception" in result:
            raise HTTPException(status_code=500, detail=f"Error en la API de Moodle: {result['exception']}")
        else:
            return {"message": "Curso actualizado exitosamente"}
    else:
        raise HTTPException(status_code=500, detail=f"Error al llamar a la API de Moodle: {response.text}")








In [6]:
"""

# Ejemplo de llamada en FastAPI
course_data = {
    "id": 3,
    "fullname": "Este es el nuevo Nombre del curso",
    "shortname": "nombre_corto",
    "categoryid": 1,
    "visible": 1,
    "format": "topics"
}
# Parsear el JSON a un objeto Python usando el modelo Pydantic
course_obj = CourseData(**course_data)

result = await actualizar_curso( course_obj)
print(result)


{'message': 'Tópico añadido exitosamente al curso'}


In [5]:
#Probar las llamadas http a la api de fastapi desde 
import json
import httpx

# Lee los datos del usuario desde el archivo JSON
with open("data/course_data.json", "r") as file:
    course_data_json = json.load(file)

# Realiza una solicitud POST a la ruta /add_user_moodle
url = "http://localhost:8000/update_course_moodle"
response = httpx.post(url, json=course_data_json)

# Imprime la respuesta del servidor
print(response.text)

INFO:     127.0.0.1:58434 - "POST /update_course_moodle HTTP/1.1" 200 OK
{"message":"Tópico añadido exitosamente al curso"}


In [2]:
#Poner en marcha el servidor en un hilo 
import nest_asyncio
nest_asyncio.apply()

from threading import Thread
import uvicorn

# Variable de bandera para controlar el hilo
stop_server = False

def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)

# Inicia el servidor en un hilo
server_thread = Thread(target=run_server)
server_thread.start()



INFO:     Started server process [11497]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
